In [0]:
# -*- coding: utf-8 -*-
"""DL4MedProject

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1XXD5I9tmQ9sTsPKQ07AaAfT2mpb8q-Uc
"""

# !python -c "import torch; print(torch.__version__)"
# !nvcc --version
# !python -c "import torch; print(torch.version.cuda)"

!pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-geometric

Looking in links: https://pytorch-geometric.com/whl/torch-1.5.0.html
  Using cached https://pytorch-geometric.com/whl/torch-1.5.0/torch_scatter-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Found existing installation: torch-scatter 2.0.4
    Uninstalling torch-scatter-2.0.4:
      Successfully uninstalled torch-scatter-2.0.4
Looking in links: https://pytorch-geometric.com/whl/torch-1.5.0.html
  Using cached https://pytorch-geometric.com/whl/torch-1.5.0/torch_sparse-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Found existing installation: torch-sparse 0.6.2
    Uninstalling torch-sparse-0.6.2:
      Successfully uninstalled torch-sparse-0.6.2
Looking in links: https://pytorch-geometric.com/whl/torch-1.5.0.html
  Using cached https://pytorch-geometric.com/whl/torch-1.5.0/torch_cluster-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Found existing installation: torch-cluster 1.5.4
    Uninstalling torch-cluster-1.5.4:
      Successfully uninstalled torch-cluster-1.5.4
Looking in links: https

In [0]:
import os.path as osp
import os
CUDA_LAUNCH_BLOCKING = 1
import torch
import random 
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import SVG
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.data import Data
from torch_geometric.data import DataLoader
from sklearn.preprocessing import LabelEncoder
from torch_geometric.nn import GraphConv, TopKPooling, GATConv, GCNConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score

%matplotlib inline

#import logging                
#logging.basicConfig(filename='what_haps.log',format='%(asctime)s %(message)s', filemode='w')
#logger=logging.getLogger()
#logger.setLevel(logging.DEBUG)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
np.random.seed(0)
torch.manual_seed(0)
data_24 = np.load('/content/drive/My Drive/imputed-normed-ep_1_24.npz', allow_pickle=True)

In [0]:
features_sequence = data_24['ep_tdata'].astype(np.float32) # 12 time series features
features_static = data_24['adm_features_all'].astype(np.float32) # 5 non-ts features,the unit of age: day
#labels = data_24["adm_labels_all"][:, 2].astype(np.float32) # 2-day mortality
#labels = data_24["adm_labels_all"][:, 1].astype(np.float32) # 1-day mortality
labels = data_24["adm_labels_all"][:, 0].astype(np.float32) # in-hospital mortality
#logger.info('Loaded 24hr data for in-hospital mortality')


In [0]:
#np.max(features_static)

trans = [0]*features_sequence.shape[0]
for i in range(features_sequence.shape[0]):
    trans[i] = features_sequence[i].T
trans= np.array(trans)

one_tenth_length = int(len(trans) * 0.1)
train_dataset_seq = trans[:one_tenth_length * 6]
val_dataset_seq = trans[one_tenth_length*6:one_tenth_length * 8]
test_dataset_seq = trans[one_tenth_length*8:]
#len(train_dataset_seq), len(val_dataset_seq), len(test_dataset_seq)

train_dataset_stat = features_static[:one_tenth_length * 6]
val_dataset_stat = features_static[one_tenth_length*6:one_tenth_length * 8]
test_dataset_stat = features_static[one_tenth_length*8:]
#len(train_dataset_stat), len(val_dataset_stat), len(test_dataset_stat)

train_dataset_label = labels[:one_tenth_length * 6]
val_dataset_label = labels[one_tenth_length*6:one_tenth_length * 8]
test_dataset_label = labels[one_tenth_length*8:]
print(len(train_dataset_label), len(val_dataset_label), len(test_dataset_label))

21654 7218 7221


In [0]:
#logger.info('Split dataset')

# upsampling
x_seq_up = []
x_static_up = []
y_up = []
for i in range(train_dataset_seq.shape[0]):
    count = 0
    if train_dataset_label[i] == 1:
        while count < 50:        #50 for 1-day, 5 for in-hosp
            x_seq_up.append(train_dataset_seq[i])
            x_static_up.append(train_dataset_stat[i])
            y_up.append(train_dataset_label[i])
            count += 1

a = np.concatenate((train_dataset_seq, x_seq_up))
b = np.concatenate((train_dataset_stat, x_static_up))
c = np.concatenate((train_dataset_label, y_up))


index = list(range(len(c)))
np.random.shuffle(index)

features_sequence = []
features_static = []
labels = []

for i in index:
    features_sequence.append(a[i])
    features_static.append(b[i])
    labels.append(c[i])

features_sequence  = np.array(features_sequence)
features_static = np.array(features_static)
labels = np.array(labels)
print(labels.shape)

#logger.info('Upsampling done')

(142004,)


In [0]:
def impute_mean(source_data, input_data):
    '''
    Takes the source data, and uses it to determine means for all
    features; it then applies them to the input data.
    
    inputs:
    source_data: a tensor to provide means
    input_data: the data to fill in NA for
    
    output:
    output_data: data with nans imputed for each feature
    
    '''
    
    output_data = input_data.copy()
    
    for feature in range(source_data.shape[1]):
        feature_mean = np.nanmean(source_data[:, feature, :][np.where(source_data[:, feature, :] != 0)])
        ind_output_data = np.where(np.isnan(output_data[:, feature, :]))
        output_data[:, feature, :][ind_output_data] =  feature_mean
    return output_data

x_seq_train_original = features_sequence.copy()
tr_features_sequence = impute_mean(x_seq_train_original, features_sequence)

tes_features_sequence = impute_mean(x_seq_train_original, test_dataset_seq)

val_features_sequence = impute_mean(x_seq_train_original, val_dataset_seq)

print(np.count_nonzero(np.isnan(tr_features_sequence)), np.count_nonzero(np.isnan(val_features_sequence)), np.count_nonzero(np.isnan(tes_features_sequence)))
#np.isnull(features_sequence)

#logger.info('Imputing done')

0 0 0


In [0]:
from sklearn.preprocessing import normalize

features_static = normalize(features_static)
val_dataset_stat = normalize(val_dataset_stat)
test_dataset_stat = normalize(test_dataset_stat)
print(np.count_nonzero(np.isnan(features_static)), np.count_nonzero(np.isnan(val_dataset_stat)), np.count_nonzero(np.isnan(test_dataset_stat)))
#logger.info('Normalized')

0 0 0


In [0]:
x=torch.from_numpy(tr_features_sequence).view(tr_features_sequence.shape[0],-1)
y=torch.from_numpy(features_static)
#print(x.shape,y.shape)
train_features=torch.cat((x, y), 1)

x = torch.from_numpy(val_features_sequence).view(val_features_sequence.shape[0],-1)
assert not torch.isnan(x).any()
y = torch.from_numpy(val_dataset_stat)
assert not torch.isnan(y).any()
val_features=torch.cat((x, y),1)

x = torch.from_numpy(tes_features_sequence).view(tes_features_sequence.shape[0],-1)
assert not torch.isnan(x).any()
y = torch.from_numpy(test_dataset_stat)
assert not torch.isnan(y).any()
test_features=torch.cat((x, y),1)
#logger.info('Combined data')
print(train_features.shape, val_features.shape, test_features.shape)
assert not torch.isnan(train_features).any()
assert not torch.isnan(val_features).any()
assert not torch.isnan(test_features).any()

torch.Size([142004, 293]) torch.Size([7218, 293]) torch.Size([7221, 293])


In [0]:
#logger.info('Preparing data loaders')

EHRdata_train = []
EHRdata_test = []
EHRdata_val = []


edges=[]    # fully connected graph between all features

for i in range(293):
  for j in range(293):
    if(i!=j):
      t=(i,j)
      t_=(j,i)
      if t not in edges and t_ not in edges:
        edges.append(t)
        edges.append(t_)
x=[]
y=[]
for i in edges:
  x.append(i[0])
  y.append(i[1])
w=[]
w.append(x)
w.append(y)
w=np.asarray(w)
edges_index = torch.tensor([w[0],w[1]], dtype=torch.long)
print(edges_index.shape)

for x,y in zip(train_features,labels):
  node_features = torch.FloatTensor(x).unsqueeze(1)
  #print(node_features.shape)
  y =torch.FloatTensor([y])
  #print('shape of y: ',y)
  data = Data(x=node_features, edge_index=edges_index, y=y)
  EHRdata_train.append(data)


for x,y in zip(val_features,val_dataset_label):
  node_features = torch.FloatTensor(x).unsqueeze(1)
  y = torch.FloatTensor([y])
  data = Data(x=node_features, edge_index=edges_index, y=y)
  EHRdata_val.append(data)

for x,y in zip(test_features,test_dataset_label):
  node_features = torch.FloatTensor(x).unsqueeze(1)
  y = torch.FloatTensor([y])
  data = Data(x=node_features, edge_index=edges_index, y=y)
  EHRdata_test.append(data)

#EHR_loader = DataLoader(EHRdata, batch_size=32)
#random.shuffle(EHRdata)


torch.Size([2, 85556])


In [0]:
batch_size= 2
train_loader = DataLoader(EHRdata_train, batch_size=batch_size)
val_loader = DataLoader(EHRdata_val, batch_size=batch_size)
test_loader = DataLoader(EHRdata_test, batch_size=batch_size)

In [0]:
#logger.info('Loaded!')

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = GCNConv(1, 256, node_dim=1)
        self.conv2 = GCNConv(256, 1, node_dim=1)
        #self.conv3 = GCNConv(128, 1, node_dim=1)
        self.pool = gap

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        print(x,x.shape)
        x = x.view(batch_size,293, -1)
        print(x,x.shape)
        x = F.relu(self.conv1(x, edge_index))
        print(x,x.shape)
        x = F.dropout(x, training=self.training)
        print(x,x.shape)
        x = x.view(batch_size,293, -1)
        x = F.relu(self.conv2(x, edge_index))
        #print(x.shape)
        #x = x.view(batch_size,293, -1)
        #x = F.relu(self.conv3(x, edge_index))
        x = x.view(293*batch_size, 1, -1)
        x = self.pool(x, batch, batch_size)
        #print(x.shape)
        x = F.sigmoid(x)
        #print(x, x.shape)
        return x

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using ',device)
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
print(model)
#logger.info('Made model')

Using  cuda
Net(
  (conv1): GCNConv(1, 256)
  (conv2): GCNConv(256, 1)
)


In [0]:

tr_acc=[]
tr_auc=[]
tr_roc=[]
va_acc=[]
va_auc=[]
va_roc=[]
te_acc=[]
te_auc=[]
te_roc=[]
loss_tr=[]
loss_val=[]


In [0]:

#to load saved model and continue training

# checkpoint = torch.load('/content/drive/My Drive/gcn_full.pth')
# start_epoch = checkpoint['epoch']
# model.load_state_dict(checkpoint['state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer'])


In [0]:

import functools, traceback
import sys

def gpu_mem_restore(func):
    "Reclaim GPU RAM if CUDA out of memory happened, or execution was interrupted"
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except:
            type, val, tb = sys.exc_info()
            traceback.clear_frames(tb)
            raise type(val).with_traceback(tb) from None
    return wrapper


import time
loss=0
@gpu_mem_restore
def train(phase):
  loss_all = 0
  if(phase=='t'):
    model.train()
    for data in train_loader:
      data = data.to(device)
      optimizer.zero_grad()
      output = model(data)
      y = torch.tensor(data.y).to(device)
      #print('Output: ',output,'\nY: ', y)
      loss = F.binary_cross_entropy(output, y)
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(),0.5)
      optimizer.step()
      print(loss, type(loss))
      loss_all += data.num_graphs * loss.item()
      #loss_all += loss.item()
      break
    loss_all = loss_all / len(train_loader)

  else:
    model.eval()
    with torch.no_grad():
      for data in val_loader:
        data = data.to(device)
        output = model(data)
        y = torch.tensor(data.y).to(device)
        loss = F.binary_cross_entropy(output, y)
        loss_all += loss.item()
        break
    loss_all = loss_all / len(val_loader)
 

  return loss_all

@gpu_mem_restore
def test(loader):
  model.eval()
  correct = 0
  predictions = []
  labels = []
  with torch.no_grad():
    for data in loader:
      data = data.to(device)
      x,e,y=data.x, data.edge_index, data.y
      print(x.shape,e.shape,y.shape)
      pred = model(data)
      print('pred: ', pred)
      pred = np.round(pred.detach().cpu()).squeeze()
      y = np.round(data.y.detach().cpu().squeeze())
      predictions.extend(pred.tolist())
      labels.extend(y.tolist())
      correct+=pred.eq(y).sum()
      break

  print('L&P: ', len(labels), labels, len(predictions), predictions)

  acc = correct/len(loader)
  #roc = roc_curve(labels, predictions)
  #auc = roc_auc_score(labels, predictions)
  #return (acc, roc, auc)
  return acc


for epoch in range(0, 50):
    start_time = time.time()
    tr_loss = train('t')
    #train_acc, roc, auc = test(train_loader)
    #train_acc = test(train_loader)
    #tr_acc.append(train_acc)
    #loss_tr.append(tr_loss)
    #tr_auc.append(auc)
    #tr_roc.append(roc)
    #val_loss = train('v')
    #val_acc = test(val_loader)
    #va_acc.append(val_acc)
    #loss_val.append(val_loss)
    #va_auc.append(auc)
    #va_roc.append(roc)
    test_acc  = test(test_loader)
    te_acc.append(test_acc)
    #te_auc.append(tes_auc)
    #te_roc.append(tes_roc)
    torch.cuda.empty_cache()
    elapse = time.strftime('%H:%M:%S', time.gmtime(int((time.time() - start_time))))
    # print('Epoch: {:03d}\n Training Loss: {:.5f} | Train Acc: {:.5f} | Validation Loss: {:.5f} | Val Acc: {:.5f} | Time: {:>9}'.
    #       format(epoch, tr_loss, train_acc, val_loss, val_acc, elapse))
    
    # state = {'epoch': epoch + 1, 'state_dict': model.state_dict(),'optimizer': optimizer.state_dict()}
    # torch.save(state, '/content/drive/My Drive/gcn_full_24hr_in-hospital.pth')
    # torch.save(model.state_dict(), '/content/drive/My Drive/gcn_model_24hr_in-hospital.pth')

# explainer = GNNExplainer(model, epochs=100)
# data = EHRdata_val[0]
# x, edges_index = data.x, data.edge_index
# node_idx=10
# node_feat_mask, edge_mask = explainer.explain_node(node_idx, x, edge_index)
# plt = explainer.visualize_subgraph(node_idx, edge_index, edge_mask, y=data.y)
# plt.show()



tensor([[  7.0000],
        [  7.0000],
        [  7.0000],
        [  7.0000],
        [  7.0000],
        [  7.0000],
        [  7.0000],
        [  7.0000],
        [  7.0000],
        [  7.0000],
        [  7.0000],
        [ 13.0000],
        [ 13.0000],
        [ 13.0000],
        [ 11.0000],
        [ 11.0000],
        [ 11.0000],
        [ 11.0000],
        [ 13.0000],
        [ 13.0000],
        [ 13.0000],
        [ 13.0000],
        [ 14.0000],
        [ 14.0000],
        [ 89.0000],
        [ 89.0000],
        [ 89.0000],
        [ 89.0000],
        [ 84.5000],
        [ 93.0000],
        [ 95.0000],
        [103.0000],
        [114.0000],
        [113.0000],
        [113.0000],
        [120.0000],
        [100.0000],
        [ 98.0000],
        [ 95.0000],
        [ 84.0000],
        [ 93.0000],
        [ 92.5000],
        [ 93.0000],
        [ 94.0000],
        [ 94.0000],
        [ 95.0000],
        [100.0000],
        [103.0000],
        [109.0000],
        [109.0000],


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1, 1])) is deprecated. Please ensure they have the same size.
/pytorch/aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


tensor([[[  7.0000],
         [  7.0000],
         [  7.0000],
         [  7.0000],
         [  7.0000],
         [  7.0000],
         [  7.0000],
         [  7.0000],
         [  7.0000],
         [  7.0000],
         [  7.0000],
         [ 13.0000],
         [ 13.0000],
         [ 13.0000],
         [ 11.0000],
         [ 11.0000],
         [ 11.0000],
         [ 11.0000],
         [ 13.0000],
         [ 13.0000],
         [ 13.0000],
         [ 13.0000],
         [ 14.0000],
         [ 14.0000],
         [ 89.0000],
         [ 89.0000],
         [ 89.0000],
         [ 89.0000],
         [ 84.5000],
         [ 93.0000],
         [ 95.0000],
         [103.0000],
         [114.0000],
         [113.0000],
         [113.0000],
         [120.0000],
         [100.0000],
         [ 98.0000],
         [ 95.0000],
         [ 84.0000],
         [ 93.0000],
         [ 92.5000],
         [ 93.0000],
         [ 94.0000],
         [ 94.0000],
         [ 95.0000],
         [100.0000],
         [103

RuntimeError: ignored

In [0]:
# device='cuda'
# for data in test_loader:
#   data = data.to(device)
#   x,e,y=data.x,data.edge_index,data.y
#   print(x,e,y)

In [0]:
# plotting for in-hospital from prev-run

#logger.info('Plotting..')
fig=plt.figure(figsize=(15,8))
plt.title("GCN Model - Training loss for In-hospital Mortality(24hrs)")
plt.xlabel("Batch")
plt.ylabel("Loss")
plt.plot(loss_p)
plt.show()
fig.savefig('/content/drive/My Drive/gcn_training_loss_24_inhsp.svg', bbox_inches='tight')

fig=plt.figure(figsize=(15,8))
plt.plot(epochs, tr_acc, 'bo', label='Training acc')
plt.plot(epochs, va_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy for In-Hospital Mortality(24hrs)')
plt.show()
fig.savefig('/content/drive/My Drive/gcn_tr_val_acc_24_inhsp.svg', bbox_inches='tight')

fig=plt.figure(figsize=(15,8))
plt.plot(tr_roc[len(tr_roc)-1][0], tr_roc[len(tr_roc)-1][1], color = 'darkorange', label = 'ROC curve\n(area = %0.2f)' % auc[len(tr_roc)-1])
plt.plot([0, 1], [0, 1], color= 'navy', linestyle = '--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('%s: ROC' % 'GCN: In-hospital Mortality(24hrs)')
plt.legend(loc = "lower right")
plt.show()
fig.savefig('/content/drive/My Drive/gcn_tr_auc_24_inhsp.svg', bbox_inches='tight')

fig=plt.figure(figsize=(15,8))
plt.plot(te_roc[len(te_roc)-1][0], te_roc[len(te_roc)-1][1], color = 'darkorange', label = 'ROC curve\n(area = %0.2f)' % auc[len(te_roc)-1])
plt.plot([0, 1], [0, 1], color= 'navy', linestyle = '--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('%s: ROC' % 'GCN: In-hospital Mortality(24hrs)')
plt.legend(loc = "lower right")
plt.show()
fig.savefig('/content/drive/My Drive/gcn_te_auc_24_inhsp.svg', bbox_inches='tight')

fig=plt.figure(figsize=(15,8))
plt.plot(va_roc[len(va_roc)-1][0], va_roc[len(va_roc)-1][1], color = 'darkorange', label = 'ROC curve\n(area = %0.2f)' % auc[len(va_roc)-1])
plt.plot([0, 1], [0, 1], color= 'navy', linestyle = '--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('%s: ROC' % 'GCN: In-hospital Mortality(24hrs)')
plt.legend(loc = "lower right")
plt.show()
fig.savefig('/content/drive/My Drive/gcn_val_auc_24_inhsp.svg', bbox_inches='tight')


fig=plt.figure(figsize=(15,8))
plt.title("GCN Model - Test Accuracy for In-hospital Mortality(24hrs)")
plt.xlabel("Batch")
plt.ylabel("Accuracy")
plt.plot(te_acc)
plt.show()
fig.savefig('/content/drive/My Drive/gcn_test_acc_24_inhsp', bbox_inches='tight')

#logger.info('Plotted!')

torch.cuda.empty_cache()